# Fast Classification with Provisioned Throughput

In [ ]:
%pip install --upgrade databricks-genai-inference datasets
dbutils.library.restartPython()

In [ ]:
# Get the API endpoint and token for the current notebook context
API_ROOT = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get() 
API_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
endpoint_name = "dl-mpt-30b-instruct"

In [ ]:
import json
import requests

data = {
    "inputs": {
        "prompt": [
            "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is Apache Spark?\n\n### Response:\n"
        ]
    },
    "params": {
        "max_tokens": 100, 
        "temperature": 0.0
    }
}
 
headers = {
    "Context-Type": "text/json",
    "Authorization": f"Bearer {API_TOKEN}"
}
 
response = requests.post(
    url=f"{API_ROOT}/serving-endpoints/{endpoint_name}/invocations",
    json=data,
    headers=headers
)
 
print(json.dumps(response.json()))

{"predictions": [{"candidates": [{"text": "Apache Spark is a multi-language engine for executing data engineering, data science, and machine learning on single-node machines or clusters.", "metadata": {"finish_reason": "stop"}}], "metadata": {"input_tokens": 35, "output_tokens": 29, "total_tokens": 64}}]}


In [ ]:
import json
import pandas as pd
import requests

# Assuming 'df' is your DataFrame and it has a column named "prompt"
# For demonstration, let's create a mock DataFrame
df = pd.DataFrame({
    "prompt": [
        "What is Apache Spark?",
        "Explain machine learning in simple terms."
    ]
})

# Convert the prompts to the required batch format
data = {
    "inputs": {
        "prompt": df['prompt'].tolist()
    },
    "params": {
        "max_tokens": 100,
        "temperature": 0.0
    }
}

headers = {
    "Content-Type": "application/json",  # Corrected header key
    "Authorization": f"Bearer {API_TOKEN}"
}

response = requests.post(
    url=f"{API_ROOT}/serving-endpoints/{endpoint_name}/invocations",
    json=data,
    headers=headers
)

print(json.dumps(response.json(), indent=2))


{
  "predictions": [
    {
      "candidates": [
        {
          "text": "\nApache Spark is an open-source, unified analytics engine for large-scale data processing. It provides high-level APIs in Scala, Java, and Python, and an optimized engine that supports general computation graphs for data analysis.\nApache Spark is a unified analytics engine for large-scale data processing. It provides high-level APIs in Scala, Java, and Python, and an optimized engine that supports general computation graphs for data analysis.\nApache Spark is an open-",
          "metadata": {
            "finish_reason": "length"
          }
        }
      ],
      "metadata": {
        "input_tokens": 5,
        "output_tokens": 100,
        "total_tokens": 105
      }
    },
    {
      "candidates": [
        {
          "text": "\nMachine learning is a field of computer science that allows computers to learn from data. It is a subfield of artificial intelligence.\nMachine learning is a field of comput

In [ ]:
from datasets import load_dataset

emotion = load_dataset("dair-ai/emotion", cache_dir="/Volumes/daniel_liden/examples/datasets")

/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:27: UserWarning: This dataset can not be stored in DBFS because either `cache_dir` or the environment variable `HF_DATASETS_CACHE` is set to a non-DBFS path. If this cluster restarts, all saved dataset information will be lost.
  warnings.warn(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-4b1fbb8a-9efc-4c53-bab7-0e6fffad98e6/lib/python3.10/site-packages/datasets/load.py:1454: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:13: UserWarning: During large dataset downloads, there could be multiple progress bar widgets that can cause performance issues for your notebook or browser. To avoid these issues, use `datasets.utils.logging.disable_progress_bar()` to turn off the progress bars.
  warnings.warn(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-4b1fbb8a-9efc-4c53-bab7-0e6fffad98e6/lib/python3.10/site-packages/datasets/load.py:1454: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
emotion

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [ ]:
prompt_template = """Your task is to classify the emotion of the provided message. Only use one of the specified emotions for your classification. The valid classifications are: JOY, SADNESS, ANGER, FEAR, LOVE, SURPRISE. No other classifications will be accepted.

### Instruction:
Read the message below and classify its emotion:

<message>{}</message>

Expected Format:
Respond with the emotion in the following format only:

<classification>EMOTION</classification>

Do not include any explanation or additional text.

### Examples:
- Message: "I was shocked by the testimony!"
  Response: <classification>SURPRISE</classification>

- Message: "I don't know how I'm ever going to move on after my team's loss..."
  Response: <classification>SADNESS</classification>

Ensure your response strictly follows the given format and only uses one of the specified emotion classifications.
### Response:\n"""

In [ ]:
# preprocess the data
emotion_messages = emotion['train']['text']

In [ ]:
import json

# List to hold the complete prompts
complete_prompts = []

# Loop through each message in the emotion_messages list
for message in emotion_messages:
    # Use the template to format the prompt with the current message
    formatted_prompt = prompt_template.format(message)
    # Add the formatted prompt to the complete_prompts list
    complete_prompts.append(formatted_prompt)


In [ ]:
complete_prompts

['Your task is to classify the emotion of the provided message. Only use one of the specified emotions for your classification. The valid classifications are: JOY, SADNESS, ANGER, FEAR, LOVE, SURPRISE. No other classifications will be accepted.\n\n### Instruction:\nRead the message below and classify its emotion:\n\n<message>i didnt feel humiliated</message>\n\nExpected Format:\nRespond with the emotion in the following format only:\n\n<classification>EMOTION</classification>\n\nDo not include any explanation or additional text.\n\n### Examples:\n- Message: "I was shocked by the testimony!"\n  Response: <classification>SURPRISE</classification>\n\n- Message: "I don\'t know how I\'m ever going to move on after my team\'s loss..."\n  Response: <classification>SADNESS</classification>\n\nEnsure your response strictly follows the given format and only uses one of the specified emotion classifications.\n### Response:\n',
 'Your task is to classify the emotion of the provided message. Only u

In [ ]:
import json
import pandas as pd
import requests
from tqdm import tqdm  # Import tqdm for progress tracking

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# Assuming complete_prompts is a list of all your prompts
# Split complete_prompts into chunks of 500
prompt_chunks = list(chunker(complete_prompts, 10))  # Convert generator to list for tqdm

responses = []  # List to store responses from each batch request

# Wrap the loop with tqdm for a progress bar
for chunk in tqdm(prompt_chunks, desc="Processing batches"):
    data = {
        "inputs": {
            "prompt": chunk
        },
        "params": {
            "max_tokens": 10,
            "temperature": 0.7
        }
    }

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_TOKEN}"
    }

    response = requests.post(
        url=f"{API_ROOT}/serving-endpoints/{endpoint_name}/invocations",
        json=data,
        headers=headers
    )

    # Check the response status and handle errors appropriately
    if response.status_code == 200:
        responses.append(response.json())
    else:
        print(f"Error processing batch: {response.status_code} - {response.text}")

# Print all responses at the end, if needed
# for i, batch_response in enumerate(responses, 1):
#     print(f"Batch {i} response:", json.dumps(batch_response, indent=2))


Processing batches:  24%|██▎       | 377/1600 [27:35<1:18:28,  3.85s/it]